In [11]:
import pandas as pd
import glob
import os
import re


In [27]:
prf = pd.read_csv(r'C:\Users\Elaine\Desktop\TCC_acidentes\data\ac_por_pessoa\acidentes2019_todas_causas_tipos.csv',encoding="ISO-8859-1", sep=";")

In [30]:
prf['estado_fisico'].value_counts()

estado_fisico
Ileso            121823
Lesões Leves     109899
Lesões Graves     36065
Não Informado     17582
Óbito             12034
Name: count, dtype: int64

In [19]:
import glob

caminho = r'data\ac_por_pessoa\*.csv'
arquivos = glob.glob(caminho)

print(arquivos)
print(len(arquivos), "arquivos encontrados")


[]
0 arquivos encontrados


In [32]:
# Caminho onde estão os arquivos de acidentes
path_acidentes = r"C:\Users\Elaine\Desktop\TCC_acidentes\data\ac_por_pessoa\*.csv"

lista_acidentes = []

for arquivo in glob.glob(path_acidentes):
    
    # Lê o CSV
    df = pd.read_csv(arquivo, encoding="ISO-8859-1", sep=";")
    
    # Filtrar colunas que você precisa
    df = df[['uf', 'br', 'km', 'municipio', 'dia_semana', 'fase_dia', 'sentido_via',
             'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo',
             'tipo_veiculo', 'data_inversa', 'estado_fisico']]
    
    # Converte data
    df["data_inversa"] = pd.to_datetime(df["data_inversa"], errors="coerce")
    
    # Extrair ano
    df["ano"] = df["data_inversa"].dt.year
    
    lista_acidentes.append(df)

# Concatena tudo em um dataframe só
prf = pd.concat(lista_acidentes, ignore_index=True)


In [33]:
path_frota = r"C:\Users\Elaine\Desktop\TCC_acidentes\data\frota\*.xls"

lista_frotas = []

def extrair_ano_primeira_linha(arquivo):
    """
    Lê apenas a primeira linha do arquivo XLS como texto
    e extrai o ano (4 dígitos) que aparece no final.
    """
    # lê SOMENTE a primeira linha como texto bruto
    df_tmp = pd.read_excel(arquivo, header=None, nrows=1, dtype=str)
    linha = str(df_tmp.iloc[0,0])

    # pega 4 dígitos no final
    ano = re.findall(r'(\d{4})', linha)
    if ano:
        return int(ano[-1])
    else:
        raise ValueError(f"Não foi possível extrair o ano do arquivo: {arquivo}")

# leitura das tabelas de frota
for arquivo in glob.glob(path_frota):

    ano = extrair_ano_primeira_linha(arquivo)

    # agora lê normalmente, pulando cabeçalho
    df = pd.read_excel(arquivo, skiprows=3)

    df = df.rename(columns={"UF": "uf", "MUNICIPIO": "municipio", "TOTAL": "frota"})
    df["ano"] = ano

    # mantém apenas o necessário
    df = df[["uf", "municipio", "frota", "ano"]]

    lista_frotas.append(df)

# concatena
frota = pd.concat(lista_frotas, ignore_index=True)


In [34]:
ac_frota = prf.merge(
    frota,
    on=["municipio", "uf", "ano"],
    how="left"
)


In [35]:
ac_frota['gravidade'] = ac_frota['estado_fisico'].apply(
    lambda x: 1 if x in ['Lesões Graves', 'Óbito'] else 0
)


In [36]:
del ac_frota['estado_fisico']

In [37]:
amostra = ac_frota.sample(frac=0.10, random_state=42)


In [38]:
amostra

,uf,br,km,municipio,dia_semana,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,tipo_veiculo,data_inversa,ano,frota,gravidade
918592,RO,364.0,"12,1",VILHENA,sexta-feira,Pleno dia,Decrescente,Céu Claro,Simples,Interseção de Vias,Sim,Motoneta,2021-07-09,2021,71697.0,0
1003548,MA,10.0,"332,2",ACAILANDIA,sexta-feira,Pleno dia,Decrescente,Céu Claro,Simples,Reta,Sim,Utilitário,2021-09-17,2021,47507.0,0
431995,PI,343.0,"341,6",TERESINA,segunda-feira,Pleno dia,Crescente,Nublado,Múltipla,Rotatória,Sim,Caminhão-trator,2020-04-27,2020,524976.0,0
452182,BA,116.0,"419,4",FEIRA DE SANTANA,sexta-feira,Anoitecer,Decrescente,Nublado,Dupla,Reta,Sim,Motocicleta,2020-05-22,2020,303018.0,0
883886,SP,116.0,211,GUARULHOS,sexta-feira,Plena Noite,Decrescente,Nublado,Múltipla,Reta,Sim,Motocicleta,2021-05-14,2021,743095.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223618,PR,476.0,141,CURITIBA,segunda-feira,Plena Noite,Crescente,Nublado,Dupla,Reta,Não,NaN,2019-09-23,2019,1595580.0,0
24001,PR,116.0,102,SAO JOSE DOS PINHAIS,quinta-feira,Pleno dia,Crescente,Céu Claro,Dupla,Reta,Sim,Semireboque,2019-01-31,2019,210951.0,0
580284,RJ,393.0,178,TRES RIOS,quinta-feira,Plena Noite,Crescente,Nublado,Simples,Curva,Sim,Caminhão-trator,2020-09-24,2020,36040.0,0
540864,MG,365.0,"372,9",PATOS DE MINAS,sábado,Pleno dia,Crescente,Chuva,Simples,Curva;Declive,Não,Semireboque,2020-08-22,2020,116891.0,0


In [40]:
amostra['gravidade'].value_counts()

gravidade
0    96523
1    18013
Name: count, dtype: int64

In [42]:
ac_frota.to_csv('../data/tratados/acidentes_frota_19-21.csv', index=False)

In [43]:
amostra.to_csv('../data/tratados/amostra_19-21.csv', index=False)

In [44]:
ac_frota.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1145355 entries, 0 to 1145354
Data columns (total 16 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   uf                      1145355 non-null  object        
 1   br                      1142804 non-null  float64       
 2   km                      1142804 non-null  object        
 3   municipio               1145355 non-null  object        
 4   dia_semana              1145355 non-null  object        
 5   fase_dia                1145355 non-null  object        
 6   sentido_via             1145355 non-null  object        
 7   condicao_metereologica  1145355 non-null  object        
 8   tipo_pista              1145355 non-null  object        
 9   tracado_via             1145355 non-null  object        
 10  uso_solo                1145355 non-null  object        
 11  tipo_veiculo            1101989 non-null  object        
 12  data_inversa  

In [45]:
ac_frota['gravidade'].value_counts(normalize=True)


gravidade
0    0.841888
1    0.158112
Name: proportion, dtype: float64